In [18]:
import pandas as pd
import qualtricsapi as qa
from pandasql import sqldf
import seaborn as sns

    
token=qa.get_secrets()


df_aw=qa.getResults(qa.SetSurveyId.survey_choices['Awareness'],token)
df_aw.rename(columns={'Verified Complete': 'Verified_Complete', 'CMS Center': 'CMS_Center'}, inplace=True)

df_BA = qa.getResults(qa.SetSurveyId.survey_choices['Business_Agility'],token)
df_BA.rename(columns={'Verified Complete': 'Verified_Complete','CMS Center': 'CMS_Center', 'Group - Which group are you joining us from?': 'Group'}, inplace=True)

df_HCD = qa.getResults(qa.SetSurveyId.survey_choices['HCD'],token)
df_HCD.rename(columns={'Verified Complete': 'Verified_Complete','CMS Center': 'CMS_Center', 'Group - Which group are you joining us from?': 'Group'}, inplace=True)


df_comp = qa.getResults(qa.SetSurveyId.survey_choices['Competency'],token)
df_comp.rename(columns={'Verified Complete': 'Verified_Complete','CMS Center': 'CMS_Center', 'Group - Which group are you joining us from?': 'Group'}, inplace=True)


data_all = pd.concat([df_aw, df_comp, df_HCD, df_BA], ignore_index=True, sort=False)


{"result":{"id":"ES_n5hsi80d55t5r57g69ekkghi6a"},"meta":{"httpStatus":"200 - OK","requestId":"0fd1dddc-1c32-47d9-9722-1da437d63990"}}
Download is 100.0 complete
{"result":{"id":"ES_d9mbil3e2kisqneh5q91cnmhod"},"meta":{"httpStatus":"200 - OK","requestId":"1d50c8b3-e9e2-4d2e-9888-2f99bffeb970"}}
Download is 100.0 complete
{"result":{"id":"ES_ps31rur0r35188uv01633bfivd"},"meta":{"httpStatus":"200 - OK","requestId":"3f8abd62-aba0-4caf-8785-3055537e4e56"}}
Download is 100.0 complete
{"result":{"id":"ES_v2qh7jo3uvkrae22mn2lknb0rs"},"meta":{"httpStatus":"200 - OK","requestId":"adbf11d1-0bcc-4c8a-b040-37dc26691ef7"}}
Download is 100.0 complete


In [ ]:
df_all = pd.DataFrame(data_all)

completed_df_all = df_all[df_all['Verified_Complete'] == 'Yes']

unique_emails = completed_df_all.drop_duplicates(subset = ['Pillar','Email'])

groupedby_df = unique_emails.groupby(['Track','Pillar']).apply(lambda x: x.reset_index(drop=True))

final_df = groupedby_df.reset_index(drop = True)

final_df

In [ ]:
df1 = pd.DataFrame(df_aw)

df1 = df1[df1['Verified_Complete'] == 'Yes']


df2 = pd.DataFrame(df_BA)

# Extract emails to exclude
emails_to_exclude = df2['Email'].tolist()

# Filter df1 to exclude these emails
filtered_df1 = df1[~df1['Email'].isin(emails_to_exclude)]

filtered_df1 = filtered_df1[['Email', 'Name']]

filtered_df1 = filtered_df1.drop_duplicates(subset='Email')

filtered_df1.to_csv('ba_spotlight_list.csv')

print(filtered_df1)

In [ ]:
def filter_emails(emails):
    # Step 1: Filter emails with Verified_Complete as 'Yes'
    verified_emails = df_aw[df_aw['Verified_Complete'] == 'Yes']
    
    # Step 2: Identify emails with Track = 'Cyber-Hygiene: Essentials' and Verified_Complete = 'Yes'
    cyber_hygiene_emails = df_aw[(df_aw['Track'] == 'Cyber-Hygiene: Essentials') & 
                                 (df_aw['Verified_Complete'] == 'Yes')]
    
    # Extract unique emails from the Cyber-Hygiene records
    cyber_hygiene_email_list = cyber_hygiene_emails['Email'].unique()
    
    # Step 3: Exclude these emails from the verified emails list
    final_emails = verified_emails[~verified_emails['Email'].isin(cyber_hygiene_email_list)]
    
    # Convert the final list of emails into a DataFrame
    result_df = pd.DataFrame(final_emails['Email'].unique(), columns=['Email'])
    
    # Return the DataFrame
    return result_df


emails = pd.DataFrame(df_aw)
result_df = filter_emails(df_aw)

# Save the result to a CSV file if needed
result_df.to_csv('filtered_emails.csv', index=False)

print(result_df)


In [19]:
data = (sqldf('''SELECT CMS_Center, Count(Distinct Email) AS Completions
            From df_aw
            Where Verified_Complete = 'Yes' AND CMS_Center != 'None'
            Group by CMS_Center
                     '''))


AttributeError: module 'pandas' has no attribute 'Dataframe'

In [ ]:
print(sqldf('''SELECT Distinct (Email), Track
            From df_aw
            Where Verified_Complete = 'Yes'
            '''));

data_aw = (sqldf('''SELECT Distinct (Email), Track
            From df_aw
            Where Verified_Complete = 'Yes'
            '''));
data_aw_df = pd.DataFrame(data_aw)

data_aw_df.to_csv('Alum-Aw.csv')

In [ ]:
#Finds learners who completed a track more than once

print(sqldf('''SELECT Email, Name, Track, Verified_Complete, Count(Email), Count(Track)
            From df_aw
            Where Verified_Complete = 'Yes'
            Group By Email, Name, Track
            Having Count(Email) > 1 AND Count(Track) > 1
                     '''));

In [ ]:
print(sqldf('''
    SELECT Distinct Email, Name, CMS_Center
    FROM df_aw
    WHERE Verified_Complete = 'Yes' AND TRIM(CMS_Center) != 'Office of Information Technology - OIT'
'''))


In [ ]:
print(sqldf('''SELECT Email, Name, Track, Verified_Complete, Level, Dropped, Count(Email), Count(Level)
            From df_comp
            Where Dropped != 'Yes'
            Group By Email, Track, Level
            Having Count(Email) > 1 AND Count(Level) > 1
                     '''));

In [ ]:
#Find duplicate enrollment

print(sqldf('''SELECT Email, Name, Track, Count(Email), Count(Track)
            From df_aw
            Where Dropped is NULL AND Verified_Complete is NULL
            Group By Email, Track
            Having Count(Email) > 1 AND Count(Track) > 1
                     '''));

In [ ]:
data_aw = sqldf('''SELECT *
            From df_aw
            Where Verified_Complete == "Yes"
                     ''')
data_aw_df = pd.DataFrame(data_aw)

data_aw_df.to_csv('Alum-Aw.csv')

data_comp = sqldf('''SELECT *
            From df_comp
            ''')
data_comp_df = pd.DataFrame(data_comp)
data_comp_df.to_csv('Alum_Comp.csv')

